In [53]:
import tensorflow as tf

# Practice tensor operations

Create a constant tensor named tensor1 containing the values [0,1,2,3,4,5,6,7] and a variable tensor named tensor2 containing the values [0,1,2,0,1,2]

In [54]:
tensor1 = tf.constant([0,1,2,3,4,5,6,7])
tensor2 = tf.Variable([0,1,2,0,1,2])
print('tensor1:', tensor1)
print('tensor2:', tensor2)

tensor1: tf.Tensor([0 1 2 3 4 5 6 7], shape=(8,), dtype=int32)
tensor2: <tf.Variable 'Variable:0' shape=(6,) dtype=int32, numpy=array([0, 1, 2, 0, 1, 2], dtype=int32)>


Reshape tensor1 so it has 2 columns and 4 rows, and tensor2 so it has 2 rows and 3 columns. Has this operation changed the nature of tensor2? How could you change it back to its former nature?

In [55]:
tensor1 = tf.reshape(tensor1, [4,2])
tensor2 = tf.reshape(tensor2, [2,3])
print('tensor1:', tensor1)
print('tensor2:', tensor2)

tensor1: tf.Tensor(
[[0 1]
 [2 3]
 [4 5]
 [6 7]], shape=(4, 2), dtype=int32)
tensor2: tf.Tensor(
[[0 1 2]
 [0 1 2]], shape=(2, 3), dtype=int32)


In [56]:
tensor2 = tf.Variable(tensor2)
print('tensor2:', tensor2)

tensor2: <tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[0, 1, 2],
       [0, 1, 2]], dtype=int32)>


Use a tensorflow function to create tensor3 with the same shape as tensor2 but filled with 1

In [57]:
tensor3 = tf.ones_like(tensor2)
print('tensor3', tensor3)

tensor3 tf.Tensor(
[[1 1 1]
 [1 1 1]], shape=(2, 3), dtype=int32)


Modify the value of tensor2 by substracting the values in tensor3, use a method so that it is an in place operation. Why would not you be able to do that with tensor1?

In [58]:
tensor2.assign_sub(tensor3)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[-1,  0,  1],
       [-1,  0,  1]], dtype=int32)>

Can you multiply tensor1 and tensor2 pointwise? How about with a matrix multiplication? Display the result of the possible operations.

In [59]:
tensor1*tensor2

InvalidArgumentError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [4,2] vs. [2,3] [Op:Mul] name: 

In [60]:
tf.matmul(tensor1, tensor2)

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[ -1,   0,   1],
       [ -5,   0,   5],
       [ -9,   0,   9],
       [-13,   0,  13]], dtype=int32)>

# Tabular data

This part of the exercise will let you deal with tabular data in order to make batch datasets ready to be fed to deep learning models.

Using the sklearn.datasets module, load the mnist dataset thanks to the load_digits function.

In [61]:
from sklearn.datasets import load_digits
digits = load_digits()

This function gives you a Data Bunch object, which works basically like a dictionnary. Create an object data containing the value of the data key and an object target containing the value of the target key.

In [62]:
data = digits.data
target = digits.target
print('data', data)
print('target', target)

data [[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
target [0 1 2 ... 8 9 8]


What is the shape of data and target? Can you understand what these objects represent using the DESCR key of the Data Bunch?

In [63]:
data.shape

(1797, 64)

In [64]:
target.shape

(1797,)

In [66]:
print(digits.DESCR)

.. _digits_dataset:

Optical recognition of handwritten digits dataset
--------------------------------------------------

**Data Set Characteristics:**

:Number of Instances: 1797
:Number of Attributes: 64
:Attribute Information: 8x8 image of integer pixels in the range 0..16.
:Missing Attribute Values: None
:Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
:Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each block. This generates
an in

Can you visualize the first image in data ?

In [67]:
from plotly import express as px

In [69]:
first_image = data[0,:].reshape([8,8])
px.imshow(first_image, color_continuous_scale='gray')

The pixel values from those images is encoded in integers between 0 and 255, it is always better to feed your deep learning models with reasonnably scaled data to avoid the network not being able to learn. To do this we'll divide the value in each pixel by 255. Do this.

In [70]:
data = data/255

Technique 1: Split the data with sklearn
Most of the time when you will be dealing with data you want to feed to a deap learning model, you will have a pandas DataFrame or numpy array at some points that contains some representation of your data and the associated values of the target variable. In those cases, it's easier to just split the data in a train and validation set using sklearn. (Remember that for very large datasets or for training and evaluating deep learning models we most of the time use the three way hold out method, where on set serves as the training set, one as the validation set to control for overfitting, and the last one is the test set against which we will evaluate the model).

Split the data and target into three different parts, one containing the train set (60%), another with the validation set (20%), and a third with the test set (20%), using sklearn.